Introduction
============

This guide provides a quick introduction to the steps required to perform a causal analysis on the data, evaluate it and generate neural networks with the results.

Running and configuration
-------------

To execute these steps, a configuration file must be provided.
Each of the steps described here can be executed independently from the others.
To facilitate that, the configuration files can be read independently from each other.
However, all the information can be included in one unique configuration file.

In [ ]:
# import sys

# argv = sys.argv[1:]
argv_causal = ['-c', 'config/quickstart/cfg_causal_analysis.yml']
argv_eval = ['-c', 'config/quickstart/cfg_evaluation.yml']
argv_nn = ['-c', 'config/quickstart/cfg_neural_networks.yml']

# 1. Causal Analysis

The first step is to perform a causal analysis on the data.

In [ ]:
from utils.setup import SetupPCAnalysis
setup_causal = SetupPCAnalysis(argv_causal)

In [ ]:
from utils.processing import proc_analysis
proc_analysis(
    setup_causal.analysis,
    setup_causal.gridpoints,
    setup_causal.spcam_inputs,
    setup_causal.spcam_outputs,
    setup_causal.cond_ind_test,
    setup_causal.ind_test_name,
    setup_causal.pc_alphas,
    setup_causal.levels,
    setup_causal.parents_idx_levs,
    setup_causal.children_idx_levs,
    setup_causal.idx_lats,
    setup_causal.idx_lons,
    setup_causal.output_file_pattern,
    setup_causal.output_folder,
    setup_causal.overwrite_pc,
    setup_causal.verbosity_pc
)

The results of the causal analysis will be available in the `output_folder` specified in the configuration.
There will be a file per each output and gridpoint.

# 2. Evaluate causal analysis
The second step is to evaluate the results achieved from the causal analysis.
As for several reasons the parents found for each variable and level can be different, a percentage threshold is used to decide which parents will be selected.
An input will be considered a parent if the percentage of gridpoints in which the causal analysis identified it as a parent is higher than the threshold.

In [ ]:
from utils.setup import SetupPCMCIAggregation
import utils.pcmci_aggregation as aggregation

setup_eval = SetupPCMCIAggregation(argv_eval)

First, the results of each output are collected from the different gridpoints.

In [ ]:
collected_results, errors = aggregation.collect_results(setup_eval)

We can check if any errors were detected while collecting the results and in which files.
These are usually missing files, for example for configuring the wrong folder or in case the causal analysis has not been done for the requested region.

In [ ]:
aggregation.print_errors(errors)

The `collected_results` can then be aggregated and filtered by threshold.

In [ ]:
aggregated_results, var_names_parents = aggregation.aggregate_results(
    collected_results, setup_eval)

The resulting object can then be examined or plotted. If necessary, adjustments can be made.

In [ ]:
aggregation.print_aggregated_results(var_names_parents, aggregated_results)

In [ ]:
# TODO This is crashing and restarting the kernel, without showing failure
aggregation.plot_aggregated_results(
    var_names_parents, aggregated_results, setup_eval)

# 3. Generate causal networks
The information obtained from the previous step can then be used to generate the causal neural networks.

There is also the possibility to generate "single neural networks", which use all inputs  and, thusly, doesn't require the causal analysis.

The models are first generated without being trained.

In [ ]:
from utils.setup import SetupNeuralNetworks

setup_nn = SetupNeuralNetworks(argv_nn)

In [ ]:
# from neural_networks.models import generate_models

# model_descriptions = generate_models(setup_nn) # This does its own evaluation

In [ ]:
from neural_networks.models import generate_all_causal_single_nn

model_descriptions = generate_all_causal_single_nn(setup_nn, aggregated_results)

In [ ]:
from neural_networks.training import train_all_models

train_all_models(model_descriptions, setup_nn)